In [1]:
# !pip install tweepy

In [ ]:
import csv
from configparser import ConfigParser
from datetime import datetime
import time
import tweepy

# Definiciones globales
MAX_TWEETS = 100  # Número máximo de tweets a obtener
QUERY = '(from:elonmusk) lang:en -is:retweet'  # Consulta ajustada

class GestorDeConfiguracion:
    """Maneja las operaciones con el archivo de configuración."""

    def __init__(self, archivo_config='config.ini'):
        self.config = ConfigParser(interpolation=None)
        read_files = self.config.read(archivo_config)
        if not read_files:
            raise FileNotFoundError(f"No se pudo encontrar el archivo de configuración: {archivo_config}")

    def obtener_credenciales(self):
        """Obtiene las credenciales de la API desde el archivo de configuración."""
        if 'twitter' not in self.config:
            raise KeyError("La sección 'twitter' no se encuentra en el archivo de configuración.")
        if 'bearer_token' not in self.config['twitter']:
            raise KeyError("El 'bearer_token' no se encuentra en la sección 'twitter' del archivo de configuración.")
        bearer_token = self.config['twitter']['bearer_token']
        return bearer_token

class ClienteTwitter:
    """Maneja la interacción con el cliente de Twitter."""

    def __init__(self, bearer_token):
        self.client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=False)

class ExtractorDeTweets:
    """Administra la extracción y almacenamiento de tweets."""

    def __init__(self, cliente, archivo_csv='tweets.csv', max_tweets=100, consulta=''):
        self.cliente = cliente
        self.archivo_csv = archivo_csv
        self.max_tweets = max_tweets
        self.consulta = consulta
        self.contador_tweets = 0
        self._inicializar_csv()

    def _inicializar_csv(self):
        """Inicializa el archivo CSV con encabezados."""
        with open(self.archivo_csv, 'w', newline='', encoding='utf-8') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(['Número de Tweet', 'Usuario', 'Texto', 'Creado en', 'Retweets', 'Likes'])

    def guardar_tweets(self):
        """Extrae y guarda tweets en el archivo CSV."""
        try:
            paginator = tweepy.Paginator(
                self.cliente.client.search_recent_tweets,
                query=self.consulta,
                tweet_fields=['created_at', 'public_metrics', 'lang'],
                expansions=['author_id'],
                max_results=100  # Máximo permitido por solicitud
            )

            with open(self.archivo_csv, 'a', newline='', encoding='utf-8') as archivo:
                escritor = csv.writer(archivo)
                for tweet_page in paginator:
                    if self.contador_tweets >= self.max_tweets:
                        break

                    if tweet_page.data is None:
                        print("No se encontraron más tweets.")
                        break

                    if 'users' in tweet_page.includes:
                        users = {u['id']: u for u in tweet_page.includes['users']}
                    else:
                        users = {}

                    for tweet in tweet_page.data:
                        self.contador_tweets += 1
                        user = users.get(tweet.author_id, {})
                        datos_tweet = [
                            self.contador_tweets,
                            user.get('username', 'N/A'),
                            tweet.text,
                            tweet.created_at,
                            tweet.public_metrics.get('retweet_count', 0),
                            tweet.public_metrics.get('like_count', 0)
                        ]
                        escritor.writerow(datos_tweet)

                        if self.contador_tweets >= self.max_tweets:
                            break

                    print(f'{datetime.now()} - Se obtuvieron {self.contador_tweets} tweets.')

                    # Espera de 2 segundos entre solicitudes para evitar alcanzar el límite de tasa
                    time.sleep(2)

        except tweepy.TooManyRequests as e:
            print("Límite de tasa alcanzado. Esperando 60 segundos antes de reintentar.")
            time.sleep(60)
            self.guardar_tweets()
        except Exception as e:
            print(f"Error al obtener tweets: {e}")

        print(f'{datetime.now()} - ¡Listo! Se obtuvieron {self.contador_tweets} tweets.')

# Ejecución principal
def main():
    gestor_config = GestorDeConfiguracion()
    bearer_token = gestor_config.obtener_credenciales()

    cliente_twitter = ClienteTwitter(bearer_token=bearer_token)

    extractor = ExtractorDeTweets(
        cliente=cliente_twitter,
        max_tweets=MAX_TWEETS,
        consulta=QUERY
    )
    extractor.guardar_tweets()

if __name__ == '__main__':
    main()


Límite de tasa alcanzado. Esperando 60 segundos antes de reintentar.
Límite de tasa alcanzado. Esperando 60 segundos antes de reintentar.
